# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

## Initializing Environment and Loading Data

In [1]:
import findspark
findspark.init('/home/shahayush954/spark-3.4.1-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kmeans_exercise').getOrCreate()

23/08/20 15:38:29 WARN Utils: Your hostname, ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/20 15:38:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/20 15:38:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/20 15:38:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
data = spark.read.csv('hack_data.csv', header=True, inferSchema=True)

In [5]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [6]:
data = data.select(['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed'])

In [7]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [8]:
data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, WPM_Typing_Speed=72.37)]

## Data Preparation and Transformation

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=data.columns, outputCol='features')

In [11]:
final_data = assembler.transform(data)

In [13]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



## KMeans with K = 2

In [14]:
from pyspark.ml.clustering import KMeans

In [15]:
kmeans2 = KMeans(k=2, featuresCol='features')

In [16]:
model_1 = kmeans2.fit(final_data)

In [17]:
results_1 = model_1.transform(final_data)

In [19]:
results_1.select(['features', 'prediction']).show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[8.0,391.09,1.0,2...|         1|
|[20.0,720.99,0.0,...|         0|
|[31.0,356.32,1.0,...|         1|
|[2.0,228.08,1.0,2...|         1|
|[20.0,408.5,0.0,3...|         1|
|[1.0,390.69,1.0,2...|         1|
|[18.0,342.97,1.0,...|         1|
|[22.0,101.61,1.0,...|         1|
|[15.0,275.53,1.0,...|         1|
|[12.0,424.83,1.0,...|         1|
|[15.0,249.09,1.0,...|         1|
|[32.0,242.48,0.0,...|         1|
|[23.0,514.54,0.0,...|         1|
|[9.0,284.77,0.0,3...|         1|
|[27.0,779.25,1.0,...|         0|
|[12.0,307.31,1.0,...|         1|
|[21.0,355.94,1.0,...|         1|
|[10.0,372.65,0.0,...|         1|
|[20.0,347.23,1.0,...|         1|
|[22.0,456.57,0.0,...|         1|
+--------------------+----------+
only showing top 20 rows



In [22]:
results_1.count()

334

In [20]:
results_1.filter('prediction = 1').count()

170

In [32]:
results_1.filter('prediction = 0').count()

164

## KMeans with K = 3

In [23]:
kmeans3 = KMeans(k=3, featuresCol='features')

In [24]:
model_2 = kmeans3.fit(final_data)

In [26]:
results_2 = model_2.transform(final_data)

In [27]:
results_2.select(['features', 'prediction']).show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[8.0,391.09,1.0,2...|         1|
|[20.0,720.99,0.0,...|         0|
|[31.0,356.32,1.0,...|         2|
|[2.0,228.08,1.0,2...|         2|
|[20.0,408.5,0.0,3...|         1|
|[1.0,390.69,1.0,2...|         1|
|[18.0,342.97,1.0,...|         2|
|[22.0,101.61,1.0,...|         2|
|[15.0,275.53,1.0,...|         2|
|[12.0,424.83,1.0,...|         1|
|[15.0,249.09,1.0,...|         2|
|[32.0,242.48,0.0,...|         2|
|[23.0,514.54,0.0,...|         1|
|[9.0,284.77,0.0,3...|         2|
|[27.0,779.25,1.0,...|         0|
|[12.0,307.31,1.0,...|         2|
|[21.0,355.94,1.0,...|         2|
|[10.0,372.65,0.0,...|         2|
|[20.0,347.23,1.0,...|         2|
|[22.0,456.57,0.0,...|         1|
+--------------------+----------+
only showing top 20 rows



In [28]:
results_2.count()

334

In [29]:
results_2.filter('prediction = 2').count()

91

In [30]:
results_2.filter('prediction = 1').count()

99

In [31]:
results_2.filter('prediction = 0').count()

144

We can see that with K = 3, the dataset is not evenly distributed, hence, it cannot be 3 hackers. However, for k = 2
the dataset was roughly divided into 2 equal groups. Hence we can say that there were 2 hackers